In [1]:
# New OS parameter to avoid warnings.  
# This will not have a material impact on your code, but prevents warnings from appearing related to new LangChain features.
import os
os.environ['USER_AGENT'] = 'RAGUserAgent'

In [2]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
import openai
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_experimental.text_splitter import SemanticChunker

In [3]:
# OpenAI Setup
from dotenv import load_dotenv
from dotenv import load_dotenv
openai.api_key = os.getenv('OPENAI_API_KEY')

In [5]:
# Load Documents
loader = WebBaseLoader(
    web_paths=("https://kbourne.github.io/chapter1.html",), 
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [6]:
docs

[Document(metadata={'source': 'https://kbourne.github.io/chapter1.html'}, page_content='\n\n      Introduction to Retrieval Augmented Generation (RAG)\n    \nDate: March 10, 2024  |  Estimated Reading Time: 15 min  |  Author: Keith Bourne\n\n  In the rapidly evolving field of artificial intelligence, Retrieval-Augmented Generation (RAG) is emerging as a significant addition to the Generative AI toolkit. RAG harnesses the strengths of Large Language Models (LLMs) and integrates them with internal data, offering a method to enhance organizational operations significantly. This book delves into the essential aspects of RAG, examining its role in augmenting the capabilities of LLMs and leveraging internal corporate data for strategic advantage.\nAs it progresses, the book outlines the potential of RAG in business, suggesting how it can make AI applications smarter, more responsive, and aligned with organizational objectives. RAG is positioned as a key facilitator of customized, efficient, 

In [7]:
# Split 
text_splitter = SemanticChunker(OpenAIEmbeddings())
splits = text_splitter.split_documents(docs)

In [10]:
text_splitter

In [12]:
splits

[Document(metadata={'source': 'https://kbourne.github.io/chapter1.html'}, page_content="\n\n      Introduction to Retrieval Augmented Generation (RAG)\n    \nDate: March 10, 2024  |  Estimated Reading Time: 15 min  |  Author: Keith Bourne\n\n  In the rapidly evolving field of artificial intelligence, Retrieval-Augmented Generation (RAG) is emerging as a significant addition to the Generative AI toolkit. RAG harnesses the strengths of Large Language Models (LLMs) and integrates them with internal data, offering a method to enhance organizational operations significantly. This book delves into the essential aspects of RAG, examining its role in augmenting the capabilities of LLMs and leveraging internal corporate data for strategic advantage. As it progresses, the book outlines the potential of RAG in business, suggesting how it can make AI applications smarter, more responsive, and aligned with organizational objectives. RAG is positioned as a key facilitator of customized, efficient, a

In [11]:
# Embed

vectorstore = Chroma.from_documents(documents=splits,embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [13]:
vectorstore

In [14]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002561BA2A160>)

In [15]:
query = "How does RAG compare with fine-turning ?"

In [16]:
relevant_docs = retriever.get_relevant_documents(query)

C:\Users\mayan\AppData\Local\Temp\ipykernel_20512\4293991353.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  relevant_docs = retriever.get_relevant_documents(query)


In [17]:
relevant_docs

[Document(metadata={'source': 'https://kbourne.github.io/chapter1.html'}, page_content='Can you imagine what you could do with all of the benefits mentioned above, but combined with all of the data within your company, about everything your company has ever done, about your customers and all of their interactions, or about all of your products and services combined with a knowledge of what a specific customer’s needs are? You do not have to imagine it, that is what RAG does! Even smaller companies are not able to access much of their internal data resources very effectively. Larger companies are swimming in petabytes of data that is not readily accessible or is not being fully utilized. Prior to RAG, most of the services you saw that connected customers or employees with the data resources of the company were really just scratching the surface of what is possible compared to if they could access ALL of the data in the company. With the advent of RAG and generative AI in general, corpor

In [18]:
# Prompt - ignore LangSmith warning, you will not need langsmith for this coding exercise
prompt = hub.pull("jclemens24/rag-prompt")

c:\Users\mayan\.conda\envs\unlock_rag\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [19]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'jclemens24', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '1a1f3ccb9a5a92363310e3b130843dfb2540239366ebe712ddd94982acc06734'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [20]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [21]:
# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [22]:
#Chain it all together with LangChain

rag_chain = (
    {"context": retriever | format_docs,
     "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
# Question - run the chain
rag_chain.invoke("What are the advantages of using RAG?")

'The advantages of using RAG (Retrieval-Augmented Generation) include:\n\n1. **Access to Comprehensive Data**: RAG allows organizations to leverage all of their internal data, including historical interactions with customers, product information, and company activities, which can lead to more informed decision-making and personalized customer interactions.\n\n2. **Enhanced Utilization of Data**: It helps companies, especially larger ones with vast amounts of data, to access and utilize their data resources more effectively, moving beyond just scratching the surface of what is possible.\n\n3. **Integration with Generative AI**: RAG combines the benefits of retrieval systems with generative AI, enabling corporations to harness the power of both technologies for improved outcomes.\n\n4. **Potential for Significant Impact**: The integration of RAG with generative AI positions organizations at the forefront of technological advancements, potentially leading to transformative changes in how 